In [1]:
%matplotlib inline
from pylab import *
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### This is a tutorial for running a saved treatment train. The steps are:
#### 1. Load treatment train. Microfiltration to Reverse Osmosis to Chlorination.
#### 2. Check and set feedwater flow
#### 3. Build and run model
#### 4. Check results
#### 5. Run with alternative cost method (TWB vs. WT3)


##### Import WaterTAP3 Package

In [2]:
import watertap as wt

##### Step 1: Load treatment train.

In [3]:
#### LOAD TRAIN ####
path = 'twb_trains/twb_mf_ro_cl2.csv'
T = wt.load_train(path)

In [4]:
# display command
wt.display.show_train(T)

##### Step 2: Check and set inlet flow

In [5]:
T.nodes['source']['Flow']

0

In [6]:
T.nodes['source']['Flow'] = 37854.118 # sets flow. units in m3/d. 37854.118 m3/d = 10 million gallons per day 

In [7]:
T.nodes['source']['Flow']

37854.118

##### Step 3:  Build and run model.

In [ ]:
M = wt.build_model(T)

In [ ]:
wt.run_model(model=M)

##### Step 5: Check results

In [ ]:
analysis_variables = ['Flow', 'TOC' , 'Nitrate', 'Cost', 'TDS']

In [ ]:
model_results = wt.post_processing.get_result_table(M, analysis_variables)

In [ ]:
model_results

In [ ]:
bar_chart = wt.post_processing.get_bar_chart(data = model_results, y_variable = 'Cost') 

In [ ]:
bar_chart = wt.post_processing.get_bar_chart(data = model_results, y_variable = 'Flow')  # TO DO BY LINK TYPE COLOR

In [ ]:
wt.post_processing.get_pipeparity(M, variable = 'levelized_cost') # million $ per m3d output

##### Step 6: Run with alternative cost method (TWB vs. WT3)

In [ ]:
M2 = wt.build_model(T, cost_method='twb')

In [ ]:
results = wt.run_model(model = M2)

In [ ]:
analysis_variables = ['Flow', 'TOC' , 'Nitrate', 'Cost', 'TDS']
model_results = wt.post_processing.get_result_table(M2, analysis_variables)
model_results

In [ ]:
wt.post_processing.get_pipeparity(M2, variable = 'levelized_cost') # million $ per m3d output

In [ ]:
# Compare the two cost methods TWB and WT3
d1 = wt.post_processing.get_bar_chart(models = [M, M2], model_names = ['wt3', 'twb'],
                                 compare_trains = True, y_variable = 'Cost')

In [ ]:
SENSE ANALYSIS FOR FLOW?!?!?!?

In [ ]:
import microfiltration_twb
import ro_twb
import chlorination_twb

In [ ]:
microfiltration_twb.cap_scaling_exp

In [ ]:
mu = 0.6
sigma = .1
num_reps = 500

In [ ]:
input_list = np.random.normal(mu,sigma, size = num_reps) #, sigma, num_reps).round(4)

In [ ]:
count, bins, ignored = plt.hist(input_list, 50, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
plt.show()

In [ ]:
def monte_run(pct_to_target1):
    print('goes in')
    cost_range_list = [];
    for value_change in pct_to_target1:
        microfiltration_twb.cap_scaling_exp = value_change
        ro_twb.cap_scaling_exp = value_change
        chlorination_twb.cap_scaling_exp = value_change
        M = wt.build_model(T)
        results = wt.run_model(model=M)
        model_results = wt.post_processing.get_result_table(M, analysis_variables)
        cost_range_list.append(model_results.Cost.sum())

    return cost_range_list

In [ ]:
from multiprocessing import Pool
import multiprocessing

In [ ]:
no_of_proc = 4
list_final = []
for i in range(no_of_proc):
    part2 = len(input_list) / no_of_proc
    i2 = ((i+1)*part2)
    list1 = input_list[int(i*part2):int(i2)]
    list_final.append(list1)

In [ ]:
pool=Pool()
dfs = pool.map(monte_run, list_final) #SomeClass().preprocess_data()

In [ ]:
joinedlist = dfs[0] + dfs[1] + dfs[2] + dfs[3]

In [ ]:
df = pd.DataFrame()
df['price'] = joinedlist
df['input_list'] = input_list

In [ ]:
figure = sns.barplot(x='input_list', y='price', data=df, color = 'blue')